# Data cleaning and visualization with pandas

This notebook will introduce you to intermediate tasks using the [`pandas`](https://pandas.pydata.org/) data analysis library and demonstrate how to dedupe, clean, group, merge/join and visualize a data set.

The data for this exercise will be a CSV of opioid deaths reported in Minnesota. Many examples here are adapted from work by Mary Jo Webster.

(If you're completely new to Python or your syntax is rusty, it might be useful to [keep this notebook open in a new tab](Python%20syntax%20cheat%20sheet.ipynb) as a reference.)

#### Session outline
- [Import pandas; load a csv into a data frame](#Import-pandas;-load-a-csv-into-a-data-frame)
- [Inspect the data](#Inspect-the-data)
- [Checking for duplicate records](#Checking-for-duplicate-records-with-groupby())
- [Renaming a column](#Renaming-a-column)
- [Recoding data with lambda()](#Recoding-data-with-lambda())
- [Cleaning up inconsistent values](#Cleaning-up-inconsistent-values)
- [Visualize the data with a bar chart](#Visualize-the-data-with-a-bar-chart)
- [Extracting year from a date](#Extracting-year-from-a-date)
- [Merging with other data (Joins)](#Merging-with-other-data)
- [Mapping TK](#Mapping-TK)

### For reference: Using Jupyter notebooks

There are many ways to write and run Python code on your computer. One way -- the method we're using today -- is to use [Jupyter notebooks](https://jupyter.org/), which run in your browser and allow you to intersperse documentation with your code. They're handy for bundling your code with a human-readable explanation of what's happening at each step. Check out some examples from the [L.A. Times](https://github.com/datadesk/notebooks) and [BuzzFeed News](https://github.com/BuzzFeedNews/everything#data-and-analyses).

**To add a new cell to your notebook**: Click the + button in the menu.

**To run a cell of code**: Select the cell and click the "Run" button in the menu, or you can press Shift+Enter.

**One common gotcha**: The notebook doesn't "know" about code you've written until you've _run_ the cell containing it. For example, if you define a variable called `my_name` in one cell, and later, when you try to access that variable in another cell but get an error that says `NameError: name 'my_name' is not defined`, the most likely solution is to run (or re-run) the cell in which you defined `my_name`.

### Import pandas; load a csv into a data frame

This was covered in more detail in the previous session, so we'll keep moving.

In [ ]:
import pandas as pd
df = pd.read_csv('data/opiate_deaths.csv')
df

### Inspect the data

Let's quickly check what columns are in our data...

In [ ]:
df.columns

... and see how many rows and columns are in the data

In [ ]:
df.shape

## Cleaning your data

When you receive data, it will almost always need some cleanup to be ready for analysis and/or display. There are some common issues to look for when you have acquired a new dataset.

- Duplicate records
- Misspellings
- Inconsistent category/variable names
- Null or empty (`''`) values that may affect calculations

### Checking for duplicate records with groupby()

Each record has a `STATEID`, which seems to imply that each row represents a unique death. But how do we know? We can use the `groupby()` method to count how many times each `STATEID` value appears. If each row is a unique person or case, there should be only 1 copy of each value.

In pandas we can chain several methods together to achieve our desired output.

Here's the full command (we'll break down the details next):

In [ ]:
df[['STATEID', 'DEATHCITY']].groupby('STATEID').count().reset_index().sort_values('DEATHCITY', ascending=False)

### Groupby, one piece at a time

First, let's run our groupby on a subset of the data:

`df[['STATEID', 'DEATHCITY']]`

This will return a view of the dataframe that only includes the STATEID and DEATHCITY columns.

Why do we even need `DEATHCITY` in this case? Because `groupby()` calls need at least two fields to work right. One or more fields that we are grouping on (`STATEID` in this case), and another field to be counted.

`df[['STATEID', 'DEATHCITY']].groupby('STATEID')`

And we'll add a `count()` call to tell pandas that we want to count the number of rows in each group, as opposed to other methods like summing or finding the average of that group.

`df[['STATEID', 'DEATHCITY']].groupby('STATEID').count()`

Next we need to include a weird pandas thing, `reset_index()` This is because otherwise, when performing a groupby, pandas will produce a MultiIndex, which is more complex than we need.

`df[['STATEID', 'DEATHCITY']].groupby('STATEID').count().reset_index()`

And finally we'll sort the results by our counted column, `DEATHCITY`. We want to surface any non-unique `STATEID`s, so we'll sort in descending order.

`df[['STATEID', 'DEATHCITY']].groupby('STATEID').count().reset_index().sort_values('DEATHCITY', ascending=False)`

Great news, the highest count is 1, meaning there are no `STATEID`s that occur more than once.

### ✍️ Your turn

Let's find out how many deaths occured in each city. Take the next few minutes to see if you can group the datafame by the `DEATHCITY` column and count the number of rows from each city.

Here's a start for you to build on:

In [ ]:
deaths_by_city = df[['DEATHCITY', 'STATEID']]  # Your code herey...

In [ ]:
# Remove before class
deaths_by_city = df[['DEATHCITY', 'STATEID']].groupby('DEATHCITY').count().reset_index().sort_values('STATEID', ascending=False)
deaths_by_city

### Renaming a column

It's confusing that the count of cities is held in a column called `STATEID`, which doesn't really have any meaning. It's always a good idea to have column headers that you can clearly understand, so let's rename the `STATEID` column in our `groupby()` to `death_count`. Note that this does NOT change the `STATEID` column in the original dataframe, as we are now working on a copy of a subset of that original data.

In [ ]:
deaths_by_city.rename(columns={'STATEID': 'death_count'}, inplace=True)
deaths_by_city

**A note on `inplace=True`:** 

Most functions that you run on your dataframes need to be assigned to variables in order to be accessed at a later time. For example, if we were to write...

`df['DEATHCITY'].drop_duplicates()`

... to get a listing of unique cities in which people died, that would not affect our dataframe `df`. If we wanted it to store that new data somewhere to access later, we would would have to write:

`unique_cities = df['DEATHCITY'].drop_duplicates()`

We could also add `inplace=True`  the function without assigning it to a variable and the dataframe will be changed:

`df['DEATHCITY'].drop_duplicates(inplace=True)`

### Recoding data with lambda()

We can also use pandas to change the data values to something more presentable or consistent. For example, let's look at the unique values for the `GENDER` column:

In [ ]:
df.GENDER.unique()

Let's change all the values in `GENDER` to the slightly more user-friendly `Male` and `Female`. We'll use a lambda function. A lambda function is shorthand way of writing what you would like to do to each row of data in this column. It's usually safer to add a new column than to change the underlying data, so we'll create a new column called `gender_clean`, and populate it with values from the `GENDER` column that we have run through our little lambda factory.

In [ ]:
df['gender_clean'] = df['GENDER'].apply(lambda x: 'Male' if x == 'M' else 'Female')
df['gender_clean']

In this case, each row was either labeled M or F, but our basic `if this, else that` logic is too naive for most real-world situations. For example, what if some rows had a value of `NA` or were null? In the above function, those rows would have gotten erroneously marked as `Female`. That's correction waiting to happen.

### A slightly more complex `lambda()`

To introduce more complex logic, we'll still use a lambda(). But instead of cramming the logic of what we want to do into one line, we'll use the lambda to apply an external function to each row.

Here's the function we'll apply to each row:

In [ ]:
def decode_gender(input):
    if input == 'M':
        return 'Male'
    elif input == 'F':
        return 'Female'
    else:
        return None

We may not have thought of every possible exception, but at least we won't wrongly assign some values to either male or female.

Now let's apply the `decode_gender` function to each row's `GENDER` column using lambda.

In [ ]:
df['gender_clean'] = df['GENDER'].apply(lambda x: decode_gender(x))
df['gender_clean'].drop_duplicates()

### Cleaning up inconsistent values

Now let's take a look at the values in the `HISPANICETHNICITY` column:

In [ ]:
df.HISPANICETHNICITY.unique()

Whoa, kind of a mess! We can't easily groupby these values right now because of the inconsistent capitalization and spelling.

One way to harmonize data in a column is to set all of the letters to uppercase. We'll start a new `hispanic_clean` column to avoid introducing errors into our original `HISPANICETHNICITY` column.

In [ ]:
df['hispanic_clean'] = df['HISPANICETHNICITY'].str.upper()
df['hispanic_clean'].unique()

OK, we cut our 8 unique values down to 5. But we still have inconsistencies in the spelling and hyphenation.

Let's deal with the simple hyphens first, by searching for and replacing instances of `NON HISPANIC`, sans hyphen. You'll notice that now we're modifying `hispanic_clean`, which we just created, directly. If we ran this against the original `HISPANICETHNICITY` column, we'd mess up some of our previous work on `hispanic_clean`.

Like most search and replace functions, the arguments passed to `replace()` are writtin the order of `1. Needle` in the `2. Haystack` you are searching through.

In [ ]:
df['hispanic_clean'] = df['hispanic_clean'].str.replace('NON HISPANIC', 'NON-HISPANIC')
df['hispanic_clean'].unique()

Better again! Now let's tackle the `NOT HISPANIC` rows, some of which have a hyphen and some that don't. We could write two lines that are similar to what we just did, but let's throw in a regular expression that can account for cases both with and without the hyphen.

`.replace(r'NOT(-| )HISPANIC', 'NON-HISPANIC')`

The `r` preceding the string tells python this is a regular expression, not a normal string.

The `(-| )` means that either a hypen or a space is an acceptable value to call a match. In either case, the command will write `NON-HISPANIC` instead.

In [ ]:
df['hispanic_clean'] = df['hispanic_clean'].str.replace(r'NOT(-| )HISPANIC', 'NON-HISPANIC')
df['hispanic_clean'].unique()

What a tidy column we have now!

### ✍️ Your turn

In the cell below, group by and count the `df` data frame:
- Get a subset of the `df` data frame
- Group by `hispanic_clean`
- Count the rows with each value
- Order descending
- Rename the count to a new variable called `death_count` after grouping

In [ ]:
# deaths_by_hispanic_ethnicity = 

In [ ]:
# Remove before class
deaths_by_hispanic_ethnicity = df[['hispanic_clean', 'STATEID']].groupby('hispanic_clean').count().reset_index().sort_values('STATEID', ascending=False)
deaths_by_hispanic_ethnicity.rename(columns={'STATEID': 'death_count'}, inplace=True)
deaths_by_hispanic_ethnicity

In our data, the vast majority of deaths are from people without Hispanic ethnicity. But Minnesota is a pretty white state, so we need to look at these numbers as a percentage, so we can compare the deaths to Minnesota's overall Hispanic population, which is about 5% of the state's total population, according to Pew Research.

We can create a new column to calculate the percentage:

In [ ]:
deaths_by_hispanic_ethnicity['pct_total'] = deaths_by_hispanic_ethnicity['death_count'] / deaths_by_hispanic_ethnicity['death_count'].sum()
deaths_by_hispanic_ethnicity

### Visualize the data with a bar chart

Pandas works well with a visualization package called [`matplotlib`](https://matplotlib.org/). There are several other visualization packages that have robust feature sets, including [Altair](https://altair-viz.github.io/), but matplotlib is already tightly integrated with pandas, so we'll start with that.

It doesn't make for the most compelling chart in the world, but once you have your data cleaned up, it's easy to make a simple bar chart:

In [ ]:
deaths_by_hispanic_ethnicity.plot.bar(x='hispanic_clean', y='death_count')

### Extracting year from a date

To look at the trend in deaths over time, we can grab just the year from the `DEATHDATE` column by taking advantage of Python's date-handling abilities. First we need to tell pandas that the `DEATHDATE` column is a `datetime`, not just an ordinary string.

In [ ]:
df['DEATHDATE'] = pd.to_datetime(df['DEATHDATE'])

Now we can use `lambda` to call `.year` on each `DEATHDATE`, and pipe it to a new column.

In [ ]:
df['death_year'] = df['DEATHDATE'].apply(lambda x: x.year)
df['death_year']

### ✍️ Your turn

In the cells below, group by and count the `df` data frame:
- Get a subset of the `df` data frame
- Group by `death_year`
- Count the rows with each value
- Order by death year
- Rename the count to a new variable called `death_count` after grouping
- Plot the death years on a bar chart

In [ ]:
# deaths_by_year =

In [ ]:
# Remove before class
deaths_by_year = df[['death_year', 'STATEID']].groupby('death_year').count().reset_index().sort_values('death_year')
deaths_by_year.rename(columns={'STATEID': 'death_count'}, inplace=True)
deaths_by_year.plot.bar(x='death_year', y='death_count')

### Merging with other data

If you remember our city `deaths_by_city` data from above, it's not surprising that Minnesota's largest city has the highest raw death count -- there are so many more people. The data would be a lot more meaningful if we could calculate the deaths per capita. But population isn't in our original CSV. But we can join our data with other data sources, provided they have some kind of identical value to join on.

First, we load a spreadsheet of population data from a reliable source from a similar time period.

In [ ]:
pop_estimates = pd.read_csv('data/mn_cities_pop_estimates.csv')
pop_estimates

Uh oh, the cities in this data set are Title Case, and the cities in our opiates data are ALL UPPERCASE.

Let's make the population csv's city names all uppercase as well:

In [ ]:
pop_estimates['city_name_capped'] = pop_estimates['City Name'].str.upper()

In [ ]:
deaths_by_city_merged = deaths_by_city.merge(
    pop_estimates,
    how="left",
    left_on="DEATHCITY",
    right_on="city_name_capped"
)
deaths_by_city_merged

### ✍️ Your turn

In the cells below:
- Create a `deaths_per_capita` field for each city by dividing `death_count` by `Total Population, 2018`
- Order the data frame by `deaths_per_capita`
- Plot the per capita deaths on a bar chart

In [ ]:
# deaths_by_city_merged['deaths_per_capita'] = 

In [ ]:
# Remove before class
deaths_by_city_merged['deaths_per_capita'] = deaths_by_city_merged['death_count'] / deaths_by_city_merged['Total Population, 2018']
deaths_by_city_merged.sort_values('deaths_per_capita', ascending=False, inplace=True)
deaths_by_city_merged['deaths_per_capita']

Whoa, that is just too many cities, and when we're doing per capita we need to be cautious about small denominators. Let's restrict the chart to only cities with 30 or more deaths.

In [ ]:
deaths_by_city_30plus = deaths_by_city_merged[deaths_by_city_merged['death_count'] > 30]
deaths_by_city_30plus.plot.bar(x='DEATHCITY', y='deaths_per_capita')

# Mapping

It's great to see these death rates on a bar chart, but a map would help us visualize this geographic dataset in another way.

First, there are a few additional libraries we need to bring in to make this all work.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify
import numpy as np
import os

Instead of city, we're going to work with counties. 

Now we want to bring in our shapefile of Minnesota counties. A shapefile is a mapping file. In this case, the Minnesota counties shapefile defines the boundary lines between different counties in the state of Minnesota. You can find your [shapefiles for your state here](https://www.census.gov/cgi-bin/geo/shapefiles/index.php).

In [ ]:
mn_cnty = gpd.read_file('gis/tl_2015_mn_county/tl_2015_mn_county.shp')

Now this looks different than how we've brought data in before. That's because plain jane pandas can't read these types of files. We need to use an additional library called geopandas.

But once you import it you can query it like any other dataframe.

In [ ]:
mn_cnty.head()

Now we need to join this shapefile with our death data so we can start to visualize where these deaths are happeneing. In order to do that, we need to find a common column to merge on.

In [ ]:
mn_cnty.columns

In [ ]:
df.columns

I know (because I added it myself) that the `GEOID` column from my `mn_cnty` shapefile is the same as the `INJURY_FIPS` column from my `df` data file. So, in order to merge these two together, I'm going to have to group by deaths data by county.

In [ ]:
deaths_by_cnty = df[['INJURY_FIPS', 'STATEID']].groupby('INJURY_FIPS').count().reset_index().sort_values('STATEID', ascending=False)
deaths_by_cnty['INJURY_FIPS'] = deaths_by_cnty['INJURY_FIPS'].astype(int).astype(str)

That second line converts the data type of our `INJURY_FIPS` field to a string. This is important because the `GEOID` field in our shapefile is stored as a string and we cannot merge two dataframes together on columns with mismatched data types.

I know that might sound like I'm speaking a different language but just tuck that knowledge away somewhere and some day you'll get an error message `ValueError: You are trying to merge on object and float64 columns.` and you'll be like... What did that woman tell me about merging on mismatched data types...

In [ ]:
deaths_by_cnty_merged = mn_cnty.merge(
    deaths_by_cnty,
    how="left",
    left_on="GEOID",
    right_on="INJURY_FIPS"
)
deaths_by_cnty_merged

Ok look at that! Merged!

Now we have to clean up our data just a touch before we can map this puppy.

First, we're going to rename that `STATEID` column because, again, it's actually our death counts column.

In [ ]:
deaths_by_cnty_merged.rename(columns={'STATEID': 'death_count'}, inplace=True)

Next we're going to replace all of the `NA` values in that death_counts column with 0 values. Because we joined the death data INTO the shapefile, it is possible that we will have counties without deaths.

And that is the case. We have 87 counties in Minnesota:

In [ ]:
len(mn_cnty)

And only 77 of those counties have death data:

In [ ]:
len(deaths_by_cnty)

Let's visualize the change. Here are all the values found in the `death_count` field in our merged dataframe:

In [ ]:
deaths_by_cnty_merged['death_count'].values

Now let's run our `fillna` function and look again:

In [ ]:
deaths_by_cnty_merged.fillna(value={'death_count': 0}, inplace=True)

In [ ]:
deaths_by_cnty_merged['death_count'].values

Look! No more `nan` values! This will allow us to visualize counties with 0 deaths on the map. If we left those as `nan` valuse, the map would omit the county all together and we'd have a swiss cheese map on our hands.

In [ ]:
#fix our map projection so it looks prettier
deaths_by_cnty_merged = deaths_by_cnty_merged.to_crs('EPSG:3395')

#establish the max and min values for our legend
vmin = deaths_by_cnty_merged.death_count.min()
vmax = deaths_by_cnty_merged.death_count.max()
fig, ax = plt.subplots(1, figsize=(10, 10))

# we don't need axes for a map
ax.axis('off')

# add a title
ax.set_title('Minnesota Deaths by County', fontdict={'fontsize': '25', 'fontweight' : '3'})

# create an annotation for the data source
ax.annotate('Source: xxxxxx', xy=(0.1, .08),  xycoords='figure fraction', 
            horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

#This bit will show us how many counties are in each bucket
print(mapclassify.JenksCaspall(deaths_by_cnty_merged.death_count, k=5))

#This bit actually builds the map
deaths_by_cnty_merged.plot(column='death_count', cmap='Greens', linewidth=0.8, ax=ax, 
                           edgecolor='0.8', scheme='JenksCaspall', legend=True)

Hoory we've made a map! But this isn't very illuminating because we've just mapped the death counts instead of death rates.

### ✍️Try your hand at creating a map using death rates for each county by:

- add county population data (look in the data folder)
- create a new dataframe by merging our death by county data to the county pop data
- calculate a new death rate column in your new dataframe
- use the death rate column to create a map

In [ ]:
# add county pop data
cnty_pop = pd.read_csv('data/mn_cnty_pop_estimates.csv')
cnty_pop['cnty_fips'] = cnty_pop['GEO_ID'].apply(lambda x: x[-5:])

# create a new dataframe
cnty_death_pop_merge = deaths_by_cnty_merged.merge(cnty_pop, how='left',
                                                   left_on='INJURY_FIPS',
                                                   right_on='cnty_fips')

# calculate death rate column per 1,000 residents
cnty_death_pop_merge['death_rate'] = (cnty_death_pop_merge['death_count'] / cnty_death_pop_merge['total_pop'])*1000

# replace those na values with 0s
cnty_death_pop_merge.fillna(value={'death_rate': 0}, inplace=True)


In [ ]:
#establish the max and min values for our legend
vmin = cnty_death_pop_merge.death_rate.min()
vmax = cnty_death_pop_merge.death_rate.max()
fig, ax = plt.subplots(1, figsize=(10, 10))

# we don't need axes for a map
ax.axis('off')

# add a title
ax.set_title('Minnesota Death Rate by County', fontdict={'fontsize': '25', 'fontweight' : '3'})

# create an annotation for the data source
ax.annotate('Source: xxxxxx', xy=(0.1, .08),  xycoords='figure fraction', 
            horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')

#This bit will show us how many counties are in each bucket
print(mapclassify.JenksCaspall(cnty_death_pop_merge.death_rate, k=5))

#This bit actually builds the map
cnty_death_pop_merge.plot(column='death_rate', cmap='Greens', linewidth=0.8, ax=ax, 
                          edgecolor='0.8', scheme='JenksCaspall', legend=True)

We can also save it as an image file!

In [ ]:
fig.savefig('img/minnesota-cnty-od-rates-map.png', dpi=300)

# If we need another example but this is too much already

### Grouping values into new categories

For a more informative bar chart, let's tag each row with a new `age_group` value. We can use `lambda()` and `apply()`, as we did above.

First, our function:

In [ ]:
def get_age_group(age):
    if age < 20:
        return '0-19'
    elif age >=20 and age < 35:
        return '20-34'
    elif age >= 35 and age < 50:
        return '35-49'
    elif age >= 50 and age < 70:
        return '50-69'
    elif age >= 70:
        return '70 and over'
    return 'Unknown'    

And now let's apply it to each row's `AGEYEARS` column

In [ ]:
df['age_group'] = df.AGEYEARS.apply(lambda x: get_age_group(x))

And be sure to check that the results make sense...

In [ ]:
df[['age_group', 'AGEYEARS']]

### ✍️ Your turn

In the cells below, group by and count the `df` data frame:
- Get a subset of the `df` data frame
- Group by `age_group`
- Count the rows with each value
- Order by age group
- Rename the count to a new variable called `death_count` after grouping
- Plot the age groups on a bar chart